# Cleaning Data

### Merging company tables

In [297]:
import pandas as pd
import re
import urllib.parse

In [298]:
df_levels = pd.read_csv("../FinalizedCSVs/companies.csv")
df_simplify = pd.read_csv("../FinalizedCSVs/simplify_companies.csv")

In [299]:
def normalize_name(name):
    if pd.isna(name):
        return ""
    name = name.lower()
    name = re.sub(r"[^a-z0-9]", "", name)  # Remove non-alphanumeric
    return name

In [300]:
df_levels["normalized_name"] = df_levels["company_slug"].apply(normalize_name)
df_simplify["normalized_name"] = df_simplify["company_name"].apply(normalize_name)

In [301]:
drop_cols = ["simplify_rating", "competitive_edge", "growth_potential", "rating_differentiation"]
df_simplify = df_simplify.drop(columns=[col for col in drop_cols if col in df_simplify.columns])

In [302]:
merged_df = pd.merge(df_levels, df_simplify, on="normalized_name", how="outer", suffixes=("_levels", "_simplify"))

### Filling in missing founded year and removing duplicate founded year column

In [303]:
merged_df["year_founded"] = merged_df["year_founded"].fillna(merged_df["founded_year"])

In [304]:
if "founded_year" in merged_df.columns:
    merged_df.drop(columns=["founded_year"], inplace=True)

In [305]:
merged_df.to_csv("../FinalizedCSVs/merged_companies.csv", index=False)

### Dropping fully empty scrapes

In [306]:
# List of columns that must not be entirely empty
necessary_columns = [
    "description", "website", "twitter", "linkedin", "year_founded",
    "num_employees", "headquarters", "simplify_take", "believer_points",
    "critic_points", "what_makes_unique", "benefits", "industries",
    "company_size", "company_stage", "total_funding", "simplify_headquarters"
]

In [307]:
filtered_companies = merged_df.dropna(subset=necessary_columns, how='all')

In [308]:
filtered_companies.reset_index(drop=True, inplace=True)

In [309]:
filtered_companies.to_csv("../FinalizedCSVs/filtered_companies.csv", index=False)

In [310]:
levels_cols = ['description', 'website', 'twitter', 'linkedin']
simplify_cols = [
    'simplify_take', 'believer_points', 'critic_points', 'what_makes_unique',
    'benefits', 'industries', 'company_size', 'company_stage',
    'total_funding', 'simplify_headquarters'
]

In [311]:
missing_levels = filtered_companies[levels_cols].isnull().all(axis=1)
num_missing_levels = missing_levels.sum()

In [312]:
missing_simplify = filtered_companies[simplify_cols].isnull().all(axis=1)
num_missing_simplify = missing_simplify.sum()

In [313]:
missing_either = (missing_levels | missing_simplify).sum()

In [314]:
print(f"Rows missing any Levels data: {num_missing_levels}")
print(f"Rows missing any Simplify data: {num_missing_simplify}")
print(f"Rows missing either Levels or Simplify data: {missing_either}")

Rows missing any Levels data: 43
Rows missing any Simplify data: 362
Rows missing either Levels or Simplify data: 405


### Normalizing industries column (atomicity)

In [315]:
# drop rows where industries is missing
df_industries = filtered_companies[["normalized_name", "industries"]].dropna()

In [316]:
# split comma-separated values into lists
df_industries["industries"] = df_industries["industries"].str.split(",")

In [317]:
# explode lists into separate rows
df_industries = df_industries.explode("industries")

In [318]:
# clean up whitespace
df_industries["industry"] = df_industries["industries"].str.strip()

In [319]:
# drop original split column
df_industries = df_industries.drop(columns=["industries"])

In [320]:
df_industries.head()

,normalized_name,industry
0,1password,Enterprise Software
0,1password,Cybersecurity
1,7shifts,Consulting
1,7shifts,Enterprise Software
2,acadianassetmanagement,Quantitative Finance


### Normalizing benefits column (atomicity)

In [321]:
# drop rows where benefits is missing
df_benefits = filtered_companies[["normalized_name", "benefits"]].dropna()

In [322]:
# split on new rows
df_benefits["benefits"] = df_benefits["benefits"].str.split("\n")

In [323]:
# explode into multiple rows
df_benefits = df_benefits.explode("benefits")

In [324]:
# clean up bullet emojis and whitespace
df_benefits["benefit"] = df_benefits["benefits"].str.replace(r"^[^\w]*", "", regex=True).str.strip()

In [325]:
# drop original list column
df_benefits = df_benefits.drop(columns=["benefits"])

In [326]:
df_benefits.head(30)

,normalized_name,benefit
0,1password,Maternity and parental leave top up programs
0,1password,Wellness spending account
0,1password,Generous PTO policy
0,1password,Company-wide wellness days off scheduled throu...
0,1password,Complimentary Headspace membership
0,1password,Comprehensive health coverage
0,1password,Employee stock option program for all full tim...
0,1password,Retirement matching program
0,1password,"Training budget, 1Password University access, ..."
0,1password,Free 1Password account (and friends and family...


### Normalize company_size and total_funding

#### Convert company_size into a categorical variable

In [327]:
filtered_companies_categorical = filtered_companies.copy()
filtered_companies_categorical["company_size_bucket"] = filtered_companies_categorical["company_size"].astype("category")

In [328]:
filtered_companies_categorical.head()

,company_slug,description,website,twitter,linkedin,year_founded,num_employees,headquarters,normalized_name,company_name,...,believer_points,critic_points,what_makes_unique,benefits,industries,company_size,company_stage,total_funding,simplify_headquarters,company_size_bucket
0,1password,"A password manager, digital vault, form filler...",https://1password.com/?ref=levels.fyi&utm_sour...,https://twitter.com/1Password,https://www.linkedin.com/company/1password,2005.0,420,4711 Yonge St 10TH FLR%2C North York%2C ON M2N...,1password,1Password,...,Growing demand for secure access solutions due...,Emerging competitors offer similar features at...,1Password integrates seamlessly with IAM syste...,👶 Maternity and parental leave top up programs...,"Enterprise Software, Cybersecurity","1,001-5,000",Series C,$920M,"Toronto, Canada","1,001-5,000"
1,7shifts,Restaurant Employee Scheduling Software. Make ...,https://www.7shifts.com/?ref=levels.fyi&utm_so...,https://twitter.com/7shifts,https://www.linkedin.com/company/7shifts,2014.0,290,701 Broadway Ave %23200%2C Saskatoon%2C SK S7N...,7shifts,7shifts,...,7shifts ranked 382 on the 2024 Deloitte Techno...,19% staff reduction may impact 7shifts' servic...,7shifts offers a comprehensive platform tailor...,Health Insurance\nCompany Equity\nFlexible Wor...,"Consulting, Enterprise Software",201-500,Series C,$131M,"Regina, Canada",201-500
2,acadian-asset-management,NaN,NaN,NaN,NaN,1986.0,NaN,NaN,acadianassetmanagement,Acadian Asset Management,...,Increased interest in ESG investing aligns wit...,Competition from firms like Two Sigma may erod...,Acadian uses sophisticated analytical models f...,Hybrid Work Options\nProfessional Development ...,"Quantitative Finance, Financial Services",201-500,NaN,NaN,"Boston, Massachusetts",201-500
3,acara-solutions,Acara Solutions is a leading provider of recru...,https://www.acarasolutions.com/?ref=levels.fyi...,https://twitter.com/Acara_Solutions,https://www.linkedin.com/company/acarasolutions,1957.0,480,500 Pearl St%2C Buffalo%2C NY 14202%2C USA,acarasolutions,Acara Solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,accenture,"Accenture plc, stylised as accenture, is an Ir...",https://www.accenture.com/?ref=levels.fyi&utm_...,https://twitter.com/Accenture,https://www.linkedin.com/company/accentureindia,1989.0,"492,185",East Building%2C Grand Canal Quay%2C Grand Can...,accenture,Accenture,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Accounting for company size showing up as a date (scrapped because it's just being corrupted when excel tries to interpret the data)

In [329]:
# filtered_companies_categorical[filtered_companies_categorical["company_size"].str.contains("Jan|Nov", na=False)]

In [330]:
# def fix_size_encoding(val):
#     if pd.isna(val): return val
#     val = str(val)
#     if val in ["10-Jan", "1-10"]:
#         return "1-10"
#     elif val in ["Nov-50", "11-50"]:
#         return "11-50"
#     else:
#         return val

# filtered_companies_categorical["company_size"] = filtered_companies_categorical["company_size"].apply(fix_size_encoding)

In [331]:
# filtered_companies_categorical.to_csv("../FinalizedCSVs/filtered_companies_test.csv", index=False)

### Convert total_funding from strings into floats

In [332]:
def clean_funding(val):
    if pd.isna(val): return None
    val = str(val).replace("$", "").replace(",", "").strip().upper()
    
    try:
        if "B" in val:
            return float(val.replace("B", "")) * 1e9
        elif "M" in val:
            return float(val.replace("M", "")) * 1e6
        elif "K" in val:
            return float(val.replace("K", "")) * 1e3
        else:
            return float(val)
    except:
        return None

In [333]:
filtered_companies_fund_and_size = filtered_companies_categorical.copy()
filtered_companies_fund_and_size["total_funding_clean"] = filtered_companies_fund_and_size["total_funding"].apply(clean_funding)


In [334]:
filtered_companies_fund_and_size.head()

,company_slug,description,website,twitter,linkedin,year_founded,num_employees,headquarters,normalized_name,company_name,...,critic_points,what_makes_unique,benefits,industries,company_size,company_stage,total_funding,simplify_headquarters,company_size_bucket,total_funding_clean
0,1password,"A password manager, digital vault, form filler...",https://1password.com/?ref=levels.fyi&utm_sour...,https://twitter.com/1Password,https://www.linkedin.com/company/1password,2005.0,420,4711 Yonge St 10TH FLR%2C North York%2C ON M2N...,1password,1Password,...,Emerging competitors offer similar features at...,1Password integrates seamlessly with IAM syste...,👶 Maternity and parental leave top up programs...,"Enterprise Software, Cybersecurity","1,001-5,000",Series C,$920M,"Toronto, Canada","1,001-5,000",920000000.0
1,7shifts,Restaurant Employee Scheduling Software. Make ...,https://www.7shifts.com/?ref=levels.fyi&utm_so...,https://twitter.com/7shifts,https://www.linkedin.com/company/7shifts,2014.0,290,701 Broadway Ave %23200%2C Saskatoon%2C SK S7N...,7shifts,7shifts,...,19% staff reduction may impact 7shifts' servic...,7shifts offers a comprehensive platform tailor...,Health Insurance\nCompany Equity\nFlexible Wor...,"Consulting, Enterprise Software",201-500,Series C,$131M,"Regina, Canada",201-500,131000000.0
2,acadian-asset-management,NaN,NaN,NaN,NaN,1986.0,NaN,NaN,acadianassetmanagement,Acadian Asset Management,...,Competition from firms like Two Sigma may erod...,Acadian uses sophisticated analytical models f...,Hybrid Work Options\nProfessional Development ...,"Quantitative Finance, Financial Services",201-500,NaN,NaN,"Boston, Massachusetts",201-500,NaN
3,acara-solutions,Acara Solutions is a leading provider of recru...,https://www.acarasolutions.com/?ref=levels.fyi...,https://twitter.com/Acara_Solutions,https://www.linkedin.com/company/acarasolutions,1957.0,480,500 Pearl St%2C Buffalo%2C NY 14202%2C USA,acarasolutions,Acara Solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,accenture,"Accenture plc, stylised as accenture, is an Ir...",https://www.accenture.com/?ref=levels.fyi&utm_...,https://twitter.com/Accenture,https://www.linkedin.com/company/accentureindia,1989.0,"492,185",East Building%2C Grand Canal Quay%2C Grand Can...,accenture,Accenture,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Create new locations table

In [335]:
# copying base df
df = filtered_companies_fund_and_size.copy()

Some issues to consider:
- Simplify only has the city, state format for US companies
- For other companies, it uses city, country

In [336]:
# building us_states set to determine whether the company is from the us

us_states = set([
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut",
    "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan",
    "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire",
    "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", 
    "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia",
    "Wisconsin", "Wyoming"
])

In [337]:
# function accounts for whether second part of simplify hq is US state

def parse_simplify_location(val):
    if pd.isna(val): return pd.Series([None, None, None])
    parts = [p.strip() for p in val.split(",")]
    
    if len(parts) == 2:
        city, region = parts
        if region in us_states:
            return pd.Series([city, region, "United States"])
        else:
            return pd.Series([city, None, region]) 
    elif len(parts) == 1:
        return pd.Series([parts[0], None, None])
    else:
        return pd.Series([None, None, None])


In [338]:
df[["city_simplify", "state_simplify", "country_simplify"]] = df["simplify_headquarters"].apply(parse_simplify_location)

In [339]:
# parse levels column to get additional info if simplify info missing

def parse_levels_location(val):
    if pd.isna(val):
        return pd.Series([None, None, None])
    
    val = urllib.parse.unquote(val)  # decode %2C into actual commas
    parts = [p.strip() for p in val.split(",")]

    if len(parts) >= 3:
        city = parts[-3]  # Usually the city is third-to-last
        state = re.findall(r"\b[A-Z]{2}\b", parts[-2])  # extract 2-letter state code
        state = state[0] if state else None
        country = parts[-1]
        return pd.Series([city, state, country])
    
    elif len(parts) == 2:
        return pd.Series([parts[0], None, parts[1]])
    else:
        return pd.Series([None, None, None])

In [340]:
df[["city_levels", "state_levels", "country_levels"]] = df["headquarters"].apply(parse_levels_location)

In [341]:
# For each company, use the city_simplify value if it exists, otherwise fall back to city_levels. Do the same for state and country.

df["final_city"] = df["city_simplify"].combine_first(df["city_levels"])
df["final_state"] = df["state_simplify"].combine_first(df["state_levels"])
df["final_country"] = df["country_simplify"].combine_first(df["country_levels"])

In [342]:
df.head(61)

,company_slug,description,website,twitter,linkedin,year_founded,num_employees,headquarters,normalized_name,company_name,...,total_funding_clean,city_simplify,state_simplify,country_simplify,city_levels,state_levels,country_levels,final_city,final_state,final_country
0,1password,"A password manager, digital vault, form filler...",https://1password.com/?ref=levels.fyi&utm_sour...,https://twitter.com/1Password,https://www.linkedin.com/company/1password,2005.0,420,4711 Yonge St 10TH FLR%2C North York%2C ON M2N...,1password,1Password,...,9.200000e+08,Toronto,None,Canada,North York,ON,Canada,Toronto,ON,Canada
1,7shifts,Restaurant Employee Scheduling Software. Make ...,https://www.7shifts.com/?ref=levels.fyi&utm_so...,https://twitter.com/7shifts,https://www.linkedin.com/company/7shifts,2014.0,290,701 Broadway Ave %23200%2C Saskatoon%2C SK S7N...,7shifts,7shifts,...,1.310000e+08,Regina,None,Canada,Saskatoon,SK,Canada,Regina,SK,Canada
2,acadian-asset-management,NaN,NaN,NaN,NaN,1986.0,NaN,NaN,acadianassetmanagement,Acadian Asset Management,...,NaN,Boston,Massachusetts,United States,None,None,None,Boston,Massachusetts,United States
3,acara-solutions,Acara Solutions is a leading provider of recru...,https://www.acarasolutions.com/?ref=levels.fyi...,https://twitter.com/Acara_Solutions,https://www.linkedin.com/company/acarasolutions,1957.0,480,500 Pearl St%2C Buffalo%2C NY 14202%2C USA,acarasolutions,Acara Solutions,...,NaN,None,None,None,Buffalo,NY,USA,Buffalo,NY,USA
4,accenture,"Accenture plc, stylised as accenture, is an Ir...",https://www.accenture.com/?ref=levels.fyi&utm_...,https://twitter.com/Accenture,https://www.linkedin.com/company/accentureindia,1989.0,"492,185",East Building%2C Grand Canal Quay%2C Grand Can...,accenture,Accenture,...,NaN,None,None,None,Co. Dublin,None,Ireland,Co. Dublin,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,ashling-partners,Ashling Partners is a professional services fi...,https://www.ashlingpartners.com/?ref=levels.fy...,https://twitter.com/AshlingPartners,https://www.linkedin.com/company/ashlingpartners,2017.0,210,1 E Wacker Dr Suite 3500%2C Chicago%2C IL 6060...,ashlingpartners,Ashling Partners,...,NaN,None,None,None,Chicago,IL,USA,Chicago,IL,USA
57,asiayo,AsiaYo is a vacation rental platform that conn...,https://asiayo.com/?ref=levels.fyi&utm_source=...,NaN,https://www.linkedin.com/company/asiayo-com,2013.0,150,18F.-3%2C No%2C No. 77%2C Section 2%2C Dunhua ...,asiayo,AsiaYo,...,NaN,None,None,None,Da’an District,None,Taiwan 106,Da’an District,None,Taiwan 106
58,asml,ASML Holding NV is a Dutch company and current...,https://www.asml.com/?ref=levels.fyi&utm_sourc...,https://twitter.com/ASMLcompany,https://www.linkedin.com/company/asml,1984.0,"24,770",De Run 6501%2C 5504 DR Veldhoven%2C Netherlands,asml,ASML,...,5.500000e+08,Veldhoven,None,Netherlands,De Run 6501,DR,Netherlands,Veldhoven,DR,Netherlands
59,assurant,"Assurant, Inc. is a global provider of risk ma...",https://www.assurant.com/?ref=levels.fyi&utm_s...,https://twitter.com/Assurant,https://www.linkedin.com/company/assurant,1892.0,"12,220",55 Broadway Suite 2901%2C New York%2C NY 10006...,assurant,Assurant,...,1.975000e+09,New York City,New York,United States,New York,NY,USA,New York City,New York,United States


#### Dealing with inconsistent abbreviations

In [343]:
us_state_map = {
    "CA": "California", "NY": "New York", "TX": "Texas", "IL": "Illinois",
    "MA": "Massachusetts", "MN": "Minnesota", "FL": "Florida", "VA": "Virginia",
    "WA": "Washington", "PA": "Pennsylvania", "OH": "Ohio", "CO": "Colorado",
    "MI": "Michigan", "NC": "North Carolina", "GA": "Georgia", "NJ": "New Jersey",
    "AZ": "Arizona", "IN": "Indiana", "OR": "Oregon", "WI": "Wisconsin",
    "DC": "District of Columbia", "MO": "Missouri", "TN": "Tennessee",
    "NV": "Nevada", "UT": "Utah", "KY": "Kentucky", "OK": "Oklahoma",
    "IA": "Iowa", "KS": "Kansas", "AL": "Alabama", "LA": "Louisiana"
}

In [344]:
country_map = {
    "USA": "United States",
    "U.S.A.": "United States",
    "United States of America": "United States",
    "UK": "United Kingdom",
    "England": "United Kingdom"
}

In [345]:
df["final_state"] = df["final_state"].replace(us_state_map)
df["final_country"] = df["final_country"].replace(country_map)

In [346]:
df.head(60)

,company_slug,description,website,twitter,linkedin,year_founded,num_employees,headquarters,normalized_name,company_name,...,total_funding_clean,city_simplify,state_simplify,country_simplify,city_levels,state_levels,country_levels,final_city,final_state,final_country
0,1password,"A password manager, digital vault, form filler...",https://1password.com/?ref=levels.fyi&utm_sour...,https://twitter.com/1Password,https://www.linkedin.com/company/1password,2005.0,420,4711 Yonge St 10TH FLR%2C North York%2C ON M2N...,1password,1Password,...,9.200000e+08,Toronto,None,Canada,North York,ON,Canada,Toronto,ON,Canada
1,7shifts,Restaurant Employee Scheduling Software. Make ...,https://www.7shifts.com/?ref=levels.fyi&utm_so...,https://twitter.com/7shifts,https://www.linkedin.com/company/7shifts,2014.0,290,701 Broadway Ave %23200%2C Saskatoon%2C SK S7N...,7shifts,7shifts,...,1.310000e+08,Regina,None,Canada,Saskatoon,SK,Canada,Regina,SK,Canada
2,acadian-asset-management,NaN,NaN,NaN,NaN,1986.0,NaN,NaN,acadianassetmanagement,Acadian Asset Management,...,NaN,Boston,Massachusetts,United States,None,None,None,Boston,Massachusetts,United States
3,acara-solutions,Acara Solutions is a leading provider of recru...,https://www.acarasolutions.com/?ref=levels.fyi...,https://twitter.com/Acara_Solutions,https://www.linkedin.com/company/acarasolutions,1957.0,480,500 Pearl St%2C Buffalo%2C NY 14202%2C USA,acarasolutions,Acara Solutions,...,NaN,None,None,None,Buffalo,NY,USA,Buffalo,New York,United States
4,accenture,"Accenture plc, stylised as accenture, is an Ir...",https://www.accenture.com/?ref=levels.fyi&utm_...,https://twitter.com/Accenture,https://www.linkedin.com/company/accentureindia,1989.0,"492,185",East Building%2C Grand Canal Quay%2C Grand Can...,accenture,Accenture,...,NaN,None,None,None,Co. Dublin,None,Ireland,Co. Dublin,None,Ireland
5,actian,NaN,NaN,NaN,NaN,2005.0,NaN,NaN,actian,Actian,...,8.450000e+07,Redwood City,California,United States,None,None,None,Redwood City,California,United States
6,activecampaign,ActiveCampaign's category-defining Customer Ex...,https://www.activecampaign.com/?ref=levels.fyi...,https://twitter.com/ActiveCampaign,https://www.linkedin.com/company/activecampaig...,2003.0,960,1 N Dearborn St 5th floor%2C Chicago%2C IL 606...,activecampaign,ActiveCampaign,...,3.600000e+08,Chicago,Illinois,United States,Chicago,IL,USA,Chicago,Illinois,United States
7,activision,"Activision Publishing, Inc. is an American vid...",https://www.activision.com/?ref=levels.fyi&utm...,https://twitter.com/Activision,https://www.linkedin.com/company/activision,1979.0,"9,250",3100 Ocean Park Blvd%2C Santa Monica%2C CA 904...,activision,Activision,...,NaN,None,None,None,Santa Monica,CA,USA,Santa Monica,California,United States
8,addepar,Addepar is a wealth management platform that s...,https://www.addepar.com/?ref=levels.fyi&utm_so...,https://twitter.com/Addepar,https://www.linkedin.com/company/addepar,2009.0,540,303 Bryant St%2C Mountain View%2C CA 94041%2C USA,addepar,Addepar,...,6.728000e+08,New York City,New York,United States,Mountain View,CA,USA,New York City,New York,United States
9,adobe,"Adobe Inc., known until October 3, 2018 as Ado...",https://www.adobe.com/?ref=levels.fyi&utm_sour...,https://twitter.com/Adobe,https://www.linkedin.com/company/adobe,1982.0,"21,250",345 Park Ave%2C San Jose%2C CA 95110%2C USA,adobe,Adobe,...,6.689000e+08,San Jose,California,United States,San Jose,CA,USA,San Jose,California,United States


In [347]:
locations_df = df[["final_city", "final_state", "final_country"]].drop_duplicates().reset_index(drop=True)
locations_df["location_id"] = locations_df.index + 1

locations_df.head(10)

,final_city,final_state,final_country,location_id
0,Toronto,ON,Canada,1
1,Regina,SK,Canada,2
2,Boston,Massachusetts,United States,3
3,Buffalo,New York,United States,4
4,Co. Dublin,None,Ireland,5
5,Redwood City,California,United States,6
6,Chicago,Illinois,United States,7
7,Santa Monica,California,United States,8
8,New York City,New York,United States,9
9,San Jose,California,United States,10


In [348]:
df = df.merge(locations_df, on=["final_city", "final_state", "final_country"], how="left")

In [349]:
df.head(10)

,company_slug,description,website,twitter,linkedin,year_founded,num_employees,headquarters,normalized_name,company_name,...,city_simplify,state_simplify,country_simplify,city_levels,state_levels,country_levels,final_city,final_state,final_country,location_id
0,1password,"A password manager, digital vault, form filler...",https://1password.com/?ref=levels.fyi&utm_sour...,https://twitter.com/1Password,https://www.linkedin.com/company/1password,2005.0,420,4711 Yonge St 10TH FLR%2C North York%2C ON M2N...,1password,1Password,...,Toronto,None,Canada,North York,ON,Canada,Toronto,ON,Canada,1
1,7shifts,Restaurant Employee Scheduling Software. Make ...,https://www.7shifts.com/?ref=levels.fyi&utm_so...,https://twitter.com/7shifts,https://www.linkedin.com/company/7shifts,2014.0,290,701 Broadway Ave %23200%2C Saskatoon%2C SK S7N...,7shifts,7shifts,...,Regina,None,Canada,Saskatoon,SK,Canada,Regina,SK,Canada,2
2,acadian-asset-management,NaN,NaN,NaN,NaN,1986.0,NaN,NaN,acadianassetmanagement,Acadian Asset Management,...,Boston,Massachusetts,United States,None,None,None,Boston,Massachusetts,United States,3
3,acara-solutions,Acara Solutions is a leading provider of recru...,https://www.acarasolutions.com/?ref=levels.fyi...,https://twitter.com/Acara_Solutions,https://www.linkedin.com/company/acarasolutions,1957.0,480,500 Pearl St%2C Buffalo%2C NY 14202%2C USA,acarasolutions,Acara Solutions,...,None,None,None,Buffalo,NY,USA,Buffalo,New York,United States,4
4,accenture,"Accenture plc, stylised as accenture, is an Ir...",https://www.accenture.com/?ref=levels.fyi&utm_...,https://twitter.com/Accenture,https://www.linkedin.com/company/accentureindia,1989.0,"492,185",East Building%2C Grand Canal Quay%2C Grand Can...,accenture,Accenture,...,None,None,None,Co. Dublin,None,Ireland,Co. Dublin,None,Ireland,5
5,actian,NaN,NaN,NaN,NaN,2005.0,NaN,NaN,actian,Actian,...,Redwood City,California,United States,None,None,None,Redwood City,California,United States,6
6,activecampaign,ActiveCampaign's category-defining Customer Ex...,https://www.activecampaign.com/?ref=levels.fyi...,https://twitter.com/ActiveCampaign,https://www.linkedin.com/company/activecampaig...,2003.0,960,1 N Dearborn St 5th floor%2C Chicago%2C IL 606...,activecampaign,ActiveCampaign,...,Chicago,Illinois,United States,Chicago,IL,USA,Chicago,Illinois,United States,7
7,activision,"Activision Publishing, Inc. is an American vid...",https://www.activision.com/?ref=levels.fyi&utm...,https://twitter.com/Activision,https://www.linkedin.com/company/activision,1979.0,"9,250",3100 Ocean Park Blvd%2C Santa Monica%2C CA 904...,activision,Activision,...,None,None,None,Santa Monica,CA,USA,Santa Monica,California,United States,8
8,addepar,Addepar is a wealth management platform that s...,https://www.addepar.com/?ref=levels.fyi&utm_so...,https://twitter.com/Addepar,https://www.linkedin.com/company/addepar,2009.0,540,303 Bryant St%2C Mountain View%2C CA 94041%2C USA,addepar,Addepar,...,New York City,New York,United States,Mountain View,CA,USA,New York City,New York,United States,9
9,adobe,"Adobe Inc., known until October 3, 2018 as Ado...",https://www.adobe.com/?ref=levels.fyi&utm_sour...,https://twitter.com/Adobe,https://www.linkedin.com/company/adobe,1982.0,"21,250",345 Park Ave%2C San Jose%2C CA 95110%2C USA,adobe,Adobe,...,San Jose,California,United States,San Jose,CA,USA,San Jose,California,United States,10


In [350]:
company_locations = df[["normalized_name", "location_id"]]

In [351]:
company_locations.head(10)

,normalized_name,location_id
0,1password,1
1,7shifts,2
2,acadianassetmanagement,3
3,acarasolutions,4
4,accenture,5
5,actian,6
6,activecampaign,7
7,activision,8
8,addepar,9
9,adobe,10


## Create Companies Table

In [352]:
df["company_id"] = df.reset_index().index + 1

In [353]:
companies_table = df[[
    "company_id", "normalized_name", "description", "website", "twitter", "linkedin",
    "year_founded", "num_employees", "total_funding_clean", "company_size", "company_stage",
    "simplify_take", "believer_points", "critic_points", "what_makes_unique"
]]

In [354]:
companies_table.head(20)

,company_id,normalized_name,description,website,twitter,linkedin,year_founded,num_employees,total_funding_clean,company_size,company_stage,simplify_take,believer_points,critic_points,what_makes_unique
0,1,1password,"A password manager, digital vault, form filler...",https://1password.com/?ref=levels.fyi&utm_sour...,https://twitter.com/1Password,https://www.linkedin.com/company/1password,2005.0,420,9.200000e+08,"1,001-5,000",Series C,What believers are saying Growing demand for s...,Growing demand for secure access solutions due...,Emerging competitors offer similar features at...,1Password integrates seamlessly with IAM syste...
1,2,7shifts,Restaurant Employee Scheduling Software. Make ...,https://www.7shifts.com/?ref=levels.fyi&utm_so...,https://twitter.com/7shifts,https://www.linkedin.com/company/7shifts,2014.0,290,1.310000e+08,201-500,Series C,What believers are saying 7shifts ranked 382 o...,7shifts ranked 382 on the 2024 Deloitte Techno...,19% staff reduction may impact 7shifts' servic...,7shifts offers a comprehensive platform tailor...
2,3,acadianassetmanagement,NaN,NaN,NaN,NaN,1986.0,NaN,NaN,201-500,NaN,What believers are saying Increased interest i...,Increased interest in ESG investing aligns wit...,Competition from firms like Two Sigma may erod...,Acadian uses sophisticated analytical models f...
3,4,acarasolutions,Acara Solutions is a leading provider of recru...,https://www.acarasolutions.com/?ref=levels.fyi...,https://twitter.com/Acara_Solutions,https://www.linkedin.com/company/acarasolutions,1957.0,480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,accenture,"Accenture plc, stylised as accenture, is an Ir...",https://www.accenture.com/?ref=levels.fyi&utm_...,https://twitter.com/Accenture,https://www.linkedin.com/company/accentureindia,1989.0,"492,185",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,actian,NaN,NaN,NaN,NaN,2005.0,NaN,8.450000e+07,201-500,Series E,What believers are saying Growing demand for h...,Growing demand for hybrid cloud solutions alig...,Decentralized data management challenges Actia...,"Actian offers a hybrid cloud data warehouse, A..."
6,7,activecampaign,ActiveCampaign's category-defining Customer Ex...,https://www.activecampaign.com/?ref=levels.fyi...,https://twitter.com/ActiveCampaign,https://www.linkedin.com/company/activecampaig...,2003.0,960,3.600000e+08,"501-1,000",Series C,What believers are saying Acquisition of Hilos...,Acquisition of Hilos expands WhatsApp automati...,Emerging AI-driven platforms may erode ActiveC...,ActiveCampaign offers 800+ pre-built automatio...
7,8,activision,"Activision Publishing, Inc. is an American vid...",https://www.activision.com/?ref=levels.fyi&utm...,https://twitter.com/Activision,https://www.linkedin.com/company/activision,1979.0,"9,250",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,addepar,Addepar is a wealth management platform that s...,https://www.addepar.com/?ref=levels.fyi&utm_so...,https://twitter.com/Addepar,https://www.linkedin.com/company/addepar,2009.0,540,6.728000e+08,"1,001-5,000",Late Stage VC,What believers are saying Addepar's partnershi...,Addepar's partnership with Corient enhances it...,Competition from fintech startups may erode Ad...,"Addepar integrates with over 100 partners, off..."
9,10,adobe,"Adobe Inc., known until October 3, 2018 as Ado...",https://www.adobe.com/?ref=levels.fyi&utm_sour...,https://twitter.com/Adobe,https://www.linkedin.com/company/adobe,1982.0,"21,250",6.689000e+08,"10,001+",IPO,What believers are saying Growing demand for A...,Growing demand for AI-driven creative tools bo...,Generative AI platforms like Writer pose a thr...,Adobe's Creative Cloud offers a comprehensive ...


### Update previous tables with company id

In [355]:
df_industries = df_industries.merge(df[["normalized_name", "company_id"]], on="normalized_name")
df_benefits = df_benefits.merge(df[["normalized_name", "company_id"]], on="normalized_name")
company_locations = company_locations.merge(df[["normalized_name", "company_id"]], on="normalized_name")


In [356]:
df_industries.head(10)

,normalized_name,industry,company_id
0,1password,Enterprise Software,1
1,1password,Cybersecurity,1
2,7shifts,Consulting,2
3,7shifts,Enterprise Software,2
4,acadianassetmanagement,Quantitative Finance,3
5,acadianassetmanagement,Financial Services,3
6,actian,Data & Analytics,6
7,actian,Enterprise Software,6
8,activecampaign,Data & Analytics,7
9,activecampaign,Enterprise Software,7


In [357]:
df_benefits.head(30)

,normalized_name,benefit,company_id
0,1password,Maternity and parental leave top up programs,1
1,1password,Wellness spending account,1
2,1password,Generous PTO policy,1
3,1password,Company-wide wellness days off scheduled throu...,1
4,1password,Complimentary Headspace membership,1
5,1password,Comprehensive health coverage,1
6,1password,Employee stock option program for all full tim...,1
7,1password,Retirement matching program,1
8,1password,"Training budget, 1Password University access, ...",1
9,1password,Free 1Password account (and friends and family...,1


In [358]:
company_locations.head(20)

,normalized_name,location_id,company_id
0,1password,1,1
1,7shifts,2,2
2,acadianassetmanagement,3,3
3,acarasolutions,4,4
4,accenture,5,5
5,actian,6,6
6,activecampaign,7,7
7,activision,8,8
8,addepar,9,9
9,adobe,10,10


### Create companies industries and industries tables

In [359]:
industries_table = pd.DataFrame({"industry": df_industries["industry"].unique()})
industries_table["industry_id"] = industries_table.reset_index().index + 1

df_industries = df_industries.merge(industries_table, on="industry")
company_industries = df_industries[["company_id", "industry_id"]]


In [360]:
industries_table.head(10)

,industry,industry_id
0,Enterprise Software,1
1,Cybersecurity,2
2,Consulting,3
3,Quantitative Finance,4
4,Financial Services,5
5,Data & Analytics,6
6,Fintech,7
7,Consumer Software,8
8,Design,9
9,Government & Public Sector,10


In [361]:
company_industries.head(10)

,company_id,industry_id
0,1,1
1,1,2
2,2,3
3,2,1
4,3,4
5,3,5
6,6,6
7,6,1
8,7,6
9,7,1


### Create benefits table

In [362]:
benefits_table = pd.DataFrame({"benefit": df_benefits["benefit"].unique()})
benefits_table["benefit_id"] = benefits_table.reset_index().index + 1

df_benefits = df_benefits.merge(benefits_table, on="benefit")
company_benefits = df_benefits[["company_id", "benefit_id"]]

In [363]:
benefits_table.head(10)

,benefit,benefit_id
0,Maternity and parental leave top up programs,1
1,Wellness spending account,2
2,Generous PTO policy,3
3,Company-wide wellness days off scheduled throu...,4
4,Complimentary Headspace membership,5
5,Comprehensive health coverage,6
6,Employee stock option program for all full tim...,7
7,Retirement matching program,8
8,"Training budget, 1Password University access, ...",9
9,Free 1Password account (and friends and family...,10


In [364]:
company_benefits.head(10)

,company_id,benefit_id
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
5,1,6
6,1,7
7,1,8
8,1,9
9,1,10


In [365]:
companies_table.to_csv("Companies.csv", index=False)
locations_df.to_csv("Locations.csv", index=False)
company_locations[["company_id", "location_id"]].to_csv("Company_Locations.csv", index=False)
industries_table.to_csv("Industries.csv", index=False)
company_industries.to_csv("Company_Industries.csv", index=False)
benefits_table.to_csv("Benefits.csv", index=False)
company_benefits.to_csv("Company_Benefits.csv", index=False)